# FarOpt Basic Example Notebook

This notebook shows how you can work with the FarOpt SDK. Full documentation can be found here https://faropt.readthedocs.io/

In [1]:
import sys
!{sys.executable} -m pip install --upgrade faropt

In [2]:
# !rm -rf  /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/faropt

### Example OR tools script for vehicle routing:

In [3]:
!mkdir src

mkdir: cannot create directory ‘src’: File exists


In [4]:
%%writefile src/main.py

"""Capacited Vehicles Routing Problem (CVRP)."""

# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Use this to publish custom metrics!
from utils import *
# [END import]


# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    # [START demands_capacities]
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data['vehicle_capacities'] = [15, 15, 15, 15]
    # [END demands_capacities]
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    log_metric('total_distance',total_distance)
    save('/tmp/main.py') # or some other saved output
    print('Total load of all routes: {}'.format(total_load))
    # [END solution_printer]


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model()
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # Create and register a transit callback.
    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # [END arc_cost]

    # Add Capacity constraint.
    # [START capacity_constraint]
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    # [END capacity_constraint]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    print('printing solutions')
    if solution:
        print_solution(data, manager, routing, solution)
    # [END print_solution]


main()

Overwriting src/main.py


### Additional note on utils included in the back end:

Notice this line in the `print_solution()` function above:

```
log_metric('total_distance',total_distance)
```

This automatically pushes the float/int value as a named metric to cloudwatch logs that you can view after the job completes, or during the job.

### Import and initialize a FarOpt object

In [5]:
from faropt import FarOpt

In [6]:
fo = FarOpt()

INFO:root:FarOpt backend is ready!
INFO:root:Async Bucket: faropt-s3asyncd0fda937-jk815bus3ob5
INFO:root:Bucket: faropt-s3bucketfbfa637e-jauzfgds5cug
INFO:root:Recipe Table: FaroptRecipeTable
INFO:root:Job table: FaroptJobTable
INFO:root:Lambda Opt function: faropt-lambdafunction2783057D4-4NK8UXFZPQBP


.... _you should see "INFO:root:FarOpt backend is ready!" if the back end is set up correctly_

# Solve Vehicle routing using FarOpt

### Configure and submit the job

_This packages the source code (main.py and any subdirectories and files)_

In [7]:
fo.configure('./src/')

INFO:root:Listing project files ...
INFO:root:Configured job!


./src/ [] ['main.py']
main.py


_....this can be a project that includes multiple folders and files, but requires a main.py at the root level_

### submit() 
Submits to Fargate, and is suitable for arbitrarily long running jobs 

In [8]:
fo.submit()

INFO:root:Submitting job
INFO:root:Submitted job! id: 2020-12-09-18-57-01-ced83a10-1ab9-4dc6-a4cc-6eb3f4fc1941


### You can check just the primary status of the job

In [9]:
fo.primary_status()

'PROVISIONING'

### .. or wait for job to complete

..._look for INFO:root:JOB COMPLETED!_

In [10]:
fo.wait()

PROVISIONING
PROVISIONING
PROVISIONING
PROVISIONING
PENDING
PENDING
PENDING
PENDING
PENDING
PENDING
PENDING
RUNNING
RUNNING
RUNNING
RUNNING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:JOB COMPLETED!


_You should see ... INFO:root:JOB COMPLETED!_

### View detailed logs from the job you ran

In [11]:
fo.logs()

INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


1607540265460 |  Starting FarOpt backend
1607540265460 | ███████╗ █████╗ ██████╗  ██████╗ ██████╗ ████████╗
1607540265460 | ██╔════╝██╔══██╗██╔══██╗██╔═══██╗██╔══██╗╚══██╔══╝
1607540265460 | █████╗  ███████║██████╔╝██║   ██║██████╔╝   ██║   
1607540265460 | ██╔══╝  ██╔══██║██╔══██╗██║   ██║██╔═══╝    ██║   
1607540265460 | ██║     ██║  ██║██║  ██║╚██████╔╝██║        ██║   
1607540265460 | ╚═╝     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝        ╚═╝   
1607540265460 | ---------------------------------------------------------------
1607540265460 | Downloading source
1607540265460 | Looking for source.zip in  faropt-s3bucketfbfa637e-jauzfgds5cug / 2020-12-09-18-57-01-ced83a10-1ab9-4dc6-a4cc-6eb3f4fc1941
1607540265460 | Note:
1607540265460 | s3bucket :  faropt-s3bucketfbfa637e-jauzfgds5cug
1607540265460 | s3key :  2020-12-09-18-57-01-ced83a10-1ab9-4dc6-a4cc-6eb3f4fc1941
1607540265460 | Downloaded source.zip! uncompressing
1607540265460 | ---------------------------------------------------------------
1

As you can see above, any output files that you save to /tmp will be automatically uploaded to the S3 bucket

```Saving /tmp/output to s3://<bucket>/path/```

### Optionally Add this problem as a standard recipe

_This is useful when you need to repeatedly run the same problem, perhaps with different data inputs_

In [15]:
fo.add_recipe(recipe_name='cvrp_problem_v126',maintainer='Lab126')

### Each recipe is given a unique ID, and this can be used to run the recipe at any time

In [21]:
fo.get_recipe_id_from_description('cvrp_problem_v126')

str

### Rerun this recipe at any time

In [23]:
fo.run_recipe(fo.get_recipe_id_from_description('cvrp_problem_v126'))

INFO:root:Downloading recipe...
INFO:root:Configured job!
INFO:root:Submitting job
INFO:root:Submitted job! id: 2020-12-09-19-18-58-3c4731c0-3dad-45ec-acdb-eae51c711ade


#### ... Or directly run using the recipe ID

In [20]:
fo.run_recipe('e4191eda-b16e-4cea-80d1-abd5f80f75ad')

INFO:root:Downloading recipe...
INFO:root:Configured job!
INFO:root:Submitting job
INFO:root:Submitted job! id: 2020-12-09-19-18-28-2dd6801e-52ef-43aa-986b-1faede889d20


## You can also run this job as a micro job (on AWS Lambda)

Note the the libraries you can use are limited to ortools, pyomo and deap libraries with default solvers


In [24]:
fo.submit(micro=True)

INFO:root:Submitting job
INFO:root:Staging job
INFO:root:Staged job! id: 2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f
INFO:root:Look for s3://faropt-s3asyncd0fda937-jk815bus3ob5/staged/2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f/source.zip
INFO:root:By submitting a micro job, you are restricted to using ortools, pyomo and deap libraries for jobs that last up to 5 minutes


START RequestId: 8c9eb972-939c-4e86-83b5-6de4176eb527 Version: $LATEST
 Starting LambdaOpt backend

███████╗ █████╗ ██████╗  ██████╗ ██████╗ ████████╗
██╔════╝██╔══██╗██╔══██╗██╔═══██╗██╔══██╗╚══██╔══╝
█████╗  ███████║██████╔╝██║   ██║██████╔╝   ██║   
██╔══╝  ██╔══██║██╔══██╗██║   ██║██╔═══╝    ██║   
██║     ██║  ██║██║  ██║╚██████╔╝██║        ██║   
╚═╝     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝        ╚═╝   


---------------------------------------------------------------
Looking for source.zip in  faropt-s3asyncd0fda937-jk815bus3ob5 / staged/2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f/source.zip
Downloaded source.zip! uncompressing
Setting env variables for micro environment
---------------------------------------------------------------
reformatting with black
utils.py
main.py
printing solutions
Route for vehicle 0:
 0 Load(0) ->  1 Load(1) ->  4 Load(5) ->  3 Load(7) ->  15 Load(15) ->  0 Load(15)
Distance of the route: 2192m
Load of the route: 15

Route for vehicle 1:
 0 Lo

_...or copy the recipe ID into the run_recipe call_

### List past jobs and recipes

In [25]:
fo.list_jobs()

jobid:2020-12-09-18-38-16-b9607d38-70fe-4986-8a6a-c5ee113770be | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-12-09-18-38-16-b9607d38-70fe-4986-8a6a-c5ee113770be/source.zip
jobid:staged/2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f | bucket:faropt-s3asyncd0fda937-jk815bus3ob5 | path:staged/2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f/source.zip
jobid:2020-12-09-19-18-58-3c4731c0-3dad-45ec-acdb-eae51c711ade | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-12-09-19-18-58-3c4731c0-3dad-45ec-acdb-eae51c711ade/source.zip
jobid:staged/2020-12-09-18-28-22-2b742b17-a9f6-402b-95b8-8d11806c21d6 | bucket:faropt-s3asyncd0fda937-jk815bus3ob5 | path:staged/2020-12-09-18-28-22-2b742b17-a9f6-402b-95b8-8d11806c21d6/source.zip
jobid:2020-12-09-18-20-24-783195c0-7c5c-4cbe-8c5e-6e3e40c0254d | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-12-09-18-20-24-783195c0-7c5c-4cbe-8c5e-6e3e40c0254d/source.zip
jobid:2020-11-28-20-13-42-a6dbec89-b26a-456e-a523-03a0

{'Items': [{'path': {'S': '2020-12-09-18-38-16-b9607d38-70fe-4986-8a6a-c5ee113770be/source.zip'},
   'jobid': {'S': '2020-12-09-18-38-16-b9607d38-70fe-4986-8a6a-c5ee113770be'},
   'bucket': {'S': 'faropt-s3bucketfbfa637e-jauzfgds5cug'}},
  {'path': {'S': 'staged/2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f/source.zip'},
   'jobid': {'S': 'staged/2020-12-09-19-19-47-ef63ad97-c34d-4c25-a2d5-cebad5881e7f'},
   'bucket': {'S': 'faropt-s3asyncd0fda937-jk815bus3ob5'}},
  {'path': {'S': '2020-12-09-19-18-58-3c4731c0-3dad-45ec-acdb-eae51c711ade/source.zip'},
   'jobid': {'S': '2020-12-09-19-18-58-3c4731c0-3dad-45ec-acdb-eae51c711ade'},
   'bucket': {'S': 'faropt-s3bucketfbfa637e-jauzfgds5cug'}},
  {'path': {'S': 'staged/2020-12-09-18-28-22-2b742b17-a9f6-402b-95b8-8d11806c21d6/source.zip'},
   'jobid': {'S': 'staged/2020-12-09-18-28-22-2b742b17-a9f6-402b-95b8-8d11806c21d6'},
   'bucket': {'S': 'faropt-s3asyncd0fda937-jk815bus3ob5'}},
  {'path': {'S': '2020-12-09-18-20-24-783195c0-7c

In [26]:
fo.list_recipes()

recipeid:e4191eda-b16e-4cea-80d1-abd5f80f75ad | bucket:faropt-s3bucketfbfa637e-jauzfgds5cug | path:2020-12-09-18-57-01-ced83a10-1ab9-4dc6-a4cc-6eb3f4fc1941/source.zip | description:cvrp_problem_v126 | maintainer:Lab126


{'Items': [{'recipeid': {'S': 'e4191eda-b16e-4cea-80d1-abd5f80f75ad'},
   'code': {'S': 'see path'},
   'maintainer': {'S': 'Lab126'},
   'bucket': {'S': 'faropt-s3bucketfbfa637e-jauzfgds5cug'},
   'description': {'S': 'cvrp_problem_v126'},
   'path': {'S': '2020-12-09-18-57-01-ced83a10-1ab9-4dc6-a4cc-6eb3f4fc1941/source.zip'}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'K6MKNIMPU7B0SDC33AGP9B2AA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 09 Dec 2020 19:20:04 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '325',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'K6MKNIMPU7B0SDC33AGP9B2AA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2176190809'},
  'RetryAttempts': 0}}

### Run a project directly from S3 (requires a source.zip in S3, with the main.py at the root)

In [27]:
fo.run_s3_job(bucket='faropt-s3bucketfbfa637e-jauzfgds5cug',key='2020-12-09-18-57-01-ced83a10-1ab9-4dc6-a4cc-6eb3f4fc1941/source.zip')

INFO:root:Downloading source...
INFO:root:Configured job!
INFO:root:Submitting job
INFO:root:Submitted job! id: 2020-12-09-19-20-34-272c60c3-cfd9-4c7b-b657-868dfb5ae529


_You can also check Fargate service for running tasks!_

In [28]:
fo.wait()

PROVISIONING
PROVISIONING
PROVISIONING
PROVISIONING
PROVISIONING
PENDING
PENDING
PENDING
PENDING
PENDING
PENDING
PENDING
RUNNING
RUNNING
RUNNING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


DEPROVISIONING


INFO:root:No running tasks. Checking completed tasks...
INFO:root:JOB COMPLETED!


In [29]:
fo.logs()

INFO:root:No running tasks. Checking completed tasks...
INFO:root:No running tasks. Checking completed tasks...


1607541675806 |  Starting FarOpt backend
1607541675806 | ███████╗ █████╗ ██████╗  ██████╗ ██████╗ ████████╗
1607541675806 | ██╔════╝██╔══██╗██╔══██╗██╔═══██╗██╔══██╗╚══██╔══╝
1607541675806 | █████╗  ███████║██████╔╝██║   ██║██████╔╝   ██║   
1607541675806 | ██╔══╝  ██╔══██║██╔══██╗██║   ██║██╔═══╝    ██║   
1607541675806 | ██║     ██║  ██║██║  ██║╚██████╔╝██║        ██║   
1607541675806 | ╚═╝     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝        ╚═╝   
1607541675806 | ---------------------------------------------------------------
1607541675806 | Downloading source
1607541675806 | Looking for source.zip in  faropt-s3bucketfbfa637e-jauzfgds5cug / 2020-12-09-19-20-34-272c60c3-cfd9-4c7b-b657-868dfb5ae529
1607541675806 | Note:
1607541675806 | s3bucket :  faropt-s3bucketfbfa637e-jauzfgds5cug
1607541675806 | s3key :  2020-12-09-19-20-34-272c60c3-cfd9-4c7b-b657-868dfb5ae529
1607541675806 | Downloaded source.zip! uncompressing
1607541675806 | ---------------------------------------------------------------
1

### Getting metrics

Using utils.py, we logged a metric called "total_distance" in while submitting the job. Let's get the publised values for this metric, for this job.

In [30]:
fo.get_metric_data(metric_name='total_distance')

{'Label': 'total_distance',
 'Datapoints': [{'Timestamp': datetime.datetime(2020, 12, 9, 19, 21, tzinfo=tzlocal()),
   'Average': 6872.0,
   'Minimum': 6872.0,
   'Maximum': 6872.0,
   'Unit': 'None'}],
 'ResponseMetadata': {'RequestId': '400a53f7-a863-4b1b-82e5-3e95a27d375d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '400a53f7-a863-4b1b-82e5-3e95a27d375d',
   'content-type': 'text/xml',
   'content-length': '565',
   'date': 'Wed, 09 Dec 2020 19:28:11 GMT'},
  'RetryAttempts': 0}}